### 4.1 데이터에서 학습한다! 

- 신경망 학습에 대해 설명
- 파이썬으로 MNIST 데이터셋의 손글씨 숫자를 학습하는 코드를 구현

##### 4.1.1 데이터 주도 학습

기계학습의 중심에는 **데이터**가 존재함

##### 4.1.2 훈련 데이터와 시험 데이터

기계학습: 우선 훈련 데이터만 사용하여 학습하면서 최적의 매개변수를 찾은 다음 시험데이터를 사용하여 앞서 훈련한 모델의 실력 평가

why 데이터를 나누는가? 아직 보지 못한 데이터로도 문제를 올바르게 풀어내는 **범용능력**을 제대로 평가하기 위해!

**오버피팅?** 한 데이터셋에만 지나치게 최적화된 상태

### 4.2 손실 함수

**손실 함수**: 신경망 성능의 '나쁨'을 나타내는 지표, 일반적으로 평균제곱오차와 교차 엔트로피 오차를 사용

##### 4.2.1 평균 제곱 오차(MSE)

각 원소의 출력(추정 값)과 정답 레이블(참 값)의 차를 제곱한 후, 그 총합

In [4]:
import numpy as np
def sum_squares_error(y,t):
    return 0.5*np.sum((y-t)**2)

In [5]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 정답은 2

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # '2'일 확률이 가장 높다고 추정

sum_squares_error(np.array(y), np.array(t))

#손실함수 출력이 작으며 정답 레이블과의 오차도 작음

0.09750000000000003

In [6]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] # '7'일 확률이 가장 높다고 추정

sum_squares_error(np.array(y), np.array(t))

0.5975

##### 5.2.2 교차 엔트로피 오차

실질적으로 정답일때의 추정의 자연로그를 계산하는 식

자연로그의 그래프와 같이 교차엔트로피 오차식도 마찬가지로 정답에 해당하는 출력이 커질수록 0에 다가가다가, 그 출력이 1일때 0이됨

정답일때의 출력이 작아질수록 오차는 커짐

In [7]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))
# why + delta? np.log()함수에 0을 입력하면 마이너스 무한대를 뜻하는 -inf가 되어 계산을 진행 할 수 없기 때문!

In [8]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 정답은 2
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # '2'일 확률이 가장 높다고 추정

cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [9]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] # '7'일 확률이 가장 높다고 추정

cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

교차 엔트로피 오차값(결과값)이 작을수록 정답에 해당하는 출력이 커지므로 앞선 MSE의 판단과 일치함

##### 4.2.3 미니 배치 학습.3 미니 배치 학습

많은 데이터를 대상으로 일일이 손실 함수를 계산하는 것은 현실적이지 않으므로 **데이터 일부만 추려 전체의 근사치로 이용**

In [10]:
import tensorflow as tf

train_data,test_data = tf.keras.datasets.mnist.load_data()
(x_train, t_train) = train_data
(x_test, t_test) = test_data

x_train = x_train.reshape(-1, 28**2) #텐서플로우의 shape를 책 예시와 같게 맞춰줌
x_test = x_test.reshape(-1, 28**2)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000,)


In [11]:
# 훈련 데이터에서 무작위로 10장만 빼냄

train_size = t_train.shape[0]
batch_size=10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [13]:
np.random.choice(60000,10)

array([55110, 42292, 43041, 26883, 24893, 56777, 44403, 22388, 58723,
         787])

이제 무작위로 선택한 이 인덱스를 이용해 미니배치를 뽑아내기만 하면 됨. 

##### 4.2.4. (배치용) 교차 엔트로피 오차 구현하기

y= 신경망의 출력, t=정답 레이블

In [14]:
def cross_entropy_error(y, t):
    if y.ndim ==1:
        t = t.reshape(1, t.size) #데이터 하나당 교차 엔트로피 오차를 구하는 경우는 데이터의 형상을 바꿔줌
        y = y.reshape(1, y.size)
        
        batch_size = y.shape[0]
        return -np.sum(t*np.log(y+1e-7)) / batch_size # 이미지 한 장당 평균의 교차 엔트로피 오차를 계산

In [15]:
#정답 레이블이 원핫 인코딩이 아닌 2나 7등의 숫자 레이블로 주어졌을 때

def cross_entropy_error(y, t):
    if y.ndim ==1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
        batch_size = y.shape[0]
        return -np.sum(np.log(y[np.arange(batch_size),t])) / batch_size

np.arange(batch_size)는 0부터 batch_size-1까지 배열을 생성

t에는 레이블이 [2,7,0,9,4]와 같이 저장되어있으므로 y[np.arange(batch_size),t] 는 각 데이터의 정답 레이블에 해당하는 신경망의 출력을 추출

##### 4.2.5 왜 손실 함수를 설정하는가? 

신경망 학습에서는 매개변수의 미분(기울기)을 계산하고, 그 미분 값을 단서로 매개변수의 값을 서서히 갱신하는 과정을 반복함

- 신경망을 학습할 때 정확도를 지표로 삼으면 매개변수의 미분이 대부분의 장소에서 0이 됨. **정확도를 지표로 삼아서는 안된다**
- 손실 함수의 값은 연속적으로 변화하므로 손실함수를 지표로 삼는다